In [1]:
1+1

2

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage
import operator

# 환경 변수
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")


# AgentState 정의
class AgentState(TypedDict):
    input: str
    chat_history: Annotated[List[BaseMessage], operator.add]
    agent_outcome: str
    intermediate_steps: List[tuple]


# 기존 도구 함수들
def add_two(x: str) -> str:
    try:
        return str(float(x) + 2)
    except Exception:
        raise ValueError("숫자만 입력해 주세요.")


def population_lookup(city: str) -> str:
    data = {
        "서울": "9,733,509명 (2025년 기준)",
        "부산": "3,400,000명 (2025년 기준)",
    }
    result = data.get(city.strip().lower())
    if result:
        return result
    else:
        raise ValueError(f"{city}의 인구 정보를 찾을 수 없습니다.")


# 도구 및 LLM 설정
tools = [
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더합니다."),
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="도시 이름 입력 시 인구 정보를 반환합니다.",
    ),
]

llm = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, openai_api_key=OPEN_API_KEY, verbose=True
)

# ReAct 프롬프트 설정
react_prompt = PromptTemplate.from_template(
    """
당신은 도움이 되는 어시스턴트입니다. 주어진 도구를 사용해서 질문에 답하세요.

사용 가능한 도구:
{tools}

도구 이름: {tool_names}

다음 형식을 사용하세요:

Question: 답변해야 할 입력 질문
Thought: 무엇을 해야 할지 생각해보세요
Action: 수행할 액션, [{tool_names}] 중 하나여야 합니다
Action Input: 액션에 대한 입력
Observation: 액션의 결과
... (이 Thought/Action/Action Input/Observation을 필요한 만큼 반복)
Thought: 이제 최종 답변을 알았습니다
Final Answer: 원래 입력 질문에 대한 최종 답변

시작하세요!

Question: {input}
Thought: {agent_scratchpad}
"""
)

# ReAct Agent 생성
react_agent = create_react_agent(llm, tools, react_prompt)


# Agent 노드 함수
def agent_node(state: AgentState) -> AgentState:
    try:
        # Agent 실행
        result = react_agent.invoke(
            {
                "input": state["input"],
                "tools": tools,
                "tool_names": [tool.name for tool in tools],
                "agent_scratchpad": "",
            }
        )

        return {
            "input": state["input"],
            "chat_history": state.get("chat_history", []),
            "agent_outcome": result,
            "intermediate_steps": state.get("intermediate_steps", []),
        }
    except Exception as e:
        # Fallback 처리
        response = llm.invoke(f"다음 질문에 답해주세요: {state['input']}")
        fallback_result = f"🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:\n{response.content}"

        return {
            "input": state["input"],
            "chat_history": state.get("chat_history", []),
            "agent_outcome": fallback_result,
            "intermediate_steps": state.get("intermediate_steps", []),
        }


# StateGraph 생성
graph = StateGraph(AgentState)
graph.add_node("agent", agent_node)

# 시작점과 종료점 설정
graph.set_entry_point("agent")
graph.add_edge("agent", END)

# 그래프 컴파일
smart_agent = graph.compile()

# 테스트
if __name__ == "__main__":
    # 도구에서 찾을 수 있는 정보
    print("=== 도구에서 찾을 수 있는 정보 ===")
    result1 = smart_agent.invoke(
        {
            "input": "서울의 인구는?",
            "chat_history": [],
            "agent_outcome": "",
            "intermediate_steps": [],
        }
    )
    print(result1["agent_outcome"])

    print("\n=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===")
    result2 = smart_agent.invoke(
        {
            "input": "대구의 인구는?",
            "chat_history": [],
            "agent_outcome": "",
            "intermediate_steps": [],
        }
    )
    print(result2["agent_outcome"])

=== 도구에서 찾을 수 있는 정보 ===
🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:
2023년 기준으로 서울의 인구는 약 9백만 명 정도입니다. 하지만 인구는 지속적으로 변동할 수 있으므로, 최신 통계를 확인하는 것이 좋습니다. 서울특별시청이나 통계청의 공식 웹사이트에서 최신 정보를 확인할 수 있습니다.

=== 도구에서 찾을 수 없는 정보 (LLM Fallback) ===
🤖 도구에서 정보를 찾지 못해 일반 지식으로 답변합니다:
대구의 인구는 2023년 기준으로 약 240만 명 정도입니다. 하지만 인구는 시간이 지남에 따라 변동할 수 있으므로, 최신 정보를 확인하는 것이 좋습니다. 대구에 대한 더 구체적인 정보가 필요하시면 말씀해 주세요!
